# 📧 Email Validation System v3.0 — Full 9-Layer Pipeline

This notebook demonstrates the **complete, production-grade email validation system** used in the CyberX Django application.

## Validation Layers
| # | Layer | Purpose |
|---|-------|---------|
| 1 | **Regex format** | Fast RFC 5322 pattern check |
| 2 | **email-validator library** | Standards-compliant syntax + normalization |
| 3 | **Disposable-domain blocklist** | 5,100+ known throwaway providers |
| 4 | **Temporary-email heuristics** | Pattern / keyword / TLD scoring |
| 5 | **Domain age (WHOIS)** | Flags freshly registered domains |
| 6 | **SPF record** | Sender Policy Framework lookup |
| 7 | **DKIM record** | DomainKeys Identified Mail public-key probe |
| 8 | **DMARC policy** | Domain-based Message Authentication check |
| 9 | **MX / DNS deliverability** | Mail-exchange record verification |

**Plus:** Weighted risk-scoring engine (0–100) and per-IP behavioral monitoring.

---

### Requirements
```
pip install email-validator dnspython python-whois requests
```

In [ ]:
# =====================================================
# Layer 1 — Regex Format Check (RFC 5322)
# =====================================================
import re

def check_format(email: str) -> dict:
    """Quick regex-based format validation with extra sanity rules."""
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
    issues = []
    is_valid = bool(re.match(pattern, email))

    if '..' in email:
        is_valid = False
        issues.append("Contains consecutive dots")
    if email.startswith(('.', '@')) or email.endswith(('.', '@')):
        is_valid = False
        issues.append("Invalid start / end character")

    return {
        'status': 'valid' if is_valid else 'invalid',
        'is_valid': is_valid,
        'message': 'Email format is valid' if is_valid else 'Invalid email format',
        'issues': issues,
    }

# Quick test
for e in ["user@example.com", "bad@email", "user..name@test.com", "@missing.com"]:
    r = check_format(e)
    icon = "✅" if r['is_valid'] else "❌"
    print(f"{icon} {e:30s} → {r['message']}")

True
False


In [ ]:
# =====================================================
# Layer 2 — Library Validation (email-validator)
# =====================================================
from email_validator import validate_email, EmailNotValidError

def check_library(email: str) -> dict:
    """Comprehensive validation using email-validator (RFC compliance + normalization)."""
    try:
        valid = validate_email(email, check_deliverability=False)
        return {
            'status': 'valid',
            'is_valid': True,
            'normalized_email': valid.email,
            'local_part': valid.local_part,
            'domain': valid.domain,
            'message': 'Passes RFC compliance validation',
        }
    except EmailNotValidError as exc:
        return {
            'status': 'invalid',
            'is_valid': False,
            'normalized_email': None,
            'domain': email.split('@')[1].lower() if '@' in email else None,
            'message': str(exc),
        }

# Quick test
for e in ["User@Gmail.COM", "test+tag@outlook.com", "bad@email"]:
    r = check_library(e)
    icon = "✅" if r['is_valid'] else "❌"
    norm = f" → {r['normalized_email']}" if r['normalized_email'] else ""
    print(f"{icon} {e:30s}{norm}  ({r['message'][:60]})")

user@example.com
Invalid: The part after the @-sign is not valid. It should have a period.
None


In [ ]:
# =====================================================
# Layer 3 — Disposable-Domain Blocklist (5,100+ domains)
# =====================================================
# The blocklist is downloaded from the community-maintained
# `disposable-email-domains` GitHub repo.  In Django we load
# from a local file; here we fetch live for demonstration.
import requests

BLOCKLIST_URL = "https://raw.githubusercontent.com/disposable-email-domains/disposable-email-domains/HEAD/disposable_email_blocklist.conf"

print("⏳ Downloading disposable-domain blocklist …")
resp = requests.get(BLOCKLIST_URL, timeout=15)
DISPOSABLE_DOMAINS = frozenset(
    line.strip().lower()
    for line in resp.text.splitlines()
    if line.strip() and not line.startswith('#')
)
print(f"✅ Loaded {len(DISPOSABLE_DOMAINS):,} disposable domains\n")


def check_blacklist(domain: str) -> dict:
    """Check domain (and parent) against the blocklist."""
    domain = domain.lower().strip()
    if domain in DISPOSABLE_DOMAINS:
        return {'is_blacklisted': True, 'confidence': 99,
                'message': f'"{domain}" is a confirmed disposable provider', 'risk_level': 'critical'}
    parts = domain.split('.')
    if len(parts) > 2:
        parent = '.'.join(parts[-2:])
        if parent in DISPOSABLE_DOMAINS:
            return {'is_blacklisted': True, 'confidence': 97,
                    'message': f'Parent "{parent}" is a disposable provider', 'risk_level': 'critical'}
    return {'is_blacklisted': False, 'confidence': 0,
            'message': f'"{domain}" is not in the blocklist', 'risk_level': 'safe'}


# Quick test
for d in ["netoiu.com", "mailinator.com", "guerrillamail.com", "gmail.com", "outlook.com"]:
    r = check_blacklist(d)
    icon = "🚫" if r['is_blacklisted'] else "✅"
    print(f"{icon} {d:25s} → {r['message']}")

True
No MX record found: The DNS query name does not exist: invalid-domain.tld.
Failed MX check.
False


In [ ]:
# =====================================================
# Layer 4 — Temporary-Email Heuristics
# =====================================================
TEMP_PATTERNS = [
    r'^temp\d*mail\d*\.(com|org|net|email)$',
    r'^\d+m(in|inute)mail\.(com|org|net)$',
    r'.*tempmail.*\.(com|org|net|email|dev|xyz)$',
    r'.*throwaway.*\.(com|org|net|email)$',
    r'.*burner.*\.(com|org|net|email|io|kiwi)$',
    r'.*disposable.*\.(com|org|net|email|ml)$',
]

SUSPICIOUS_TLDS = {'.tk', '.ml', '.ga', '.cf', '.gq', '.pw', '.cc', '.top', '.click'}

TEMP_KEYWORDS = [
    'temp', 'temporary', 'disposable', 'throwaway', 'burner', 'fake',
    'trash', 'spam', 'guerrilla', 'anonymous', 'ephemeral', 'instant',
]


def check_temp_heuristics(domain: str) -> dict:
    """Pattern, keyword, suspicious-TLD, and heuristic scoring."""
    domain = domain.lower().strip()

    # Pattern matching
    for pat in TEMP_PATTERNS:
        if re.match(pat, domain):
            return {'is_temporary': True, 'method': 'pattern', 'confidence': 85,
                    'message': f'Matches temp-email pattern', 'risk_level': 'high'}

    # Keyword analysis
    domain_parts = re.split(r'[.\-]', domain)
    found = [kw for kw in TEMP_KEYWORDS if any(kw in p for p in domain_parts)]
    if found:
        return {'is_temporary': True, 'method': 'keyword', 'confidence': 75,
                'message': f'Contains temp keywords: {", ".join(found)}', 'risk_level': 'medium'}

    # Suspicious TLD
    tld = '.' + domain.split('.')[-1] if '.' in domain else ''
    if tld in SUSPICIOUS_TLDS:
        return {'is_temporary': True, 'method': 'suspicious_tld', 'confidence': 60,
                'message': f'Uses suspicious TLD "{tld}"', 'risk_level': 'medium'}

    return {'is_temporary': False, 'method': 'clean', 'confidence': 0,
            'message': f'No temp-email characteristics detected', 'risk_level': 'safe'}


# Quick test
for d in ["tempmail.xyz", "burneremail.com", "my-fake-inbox.tk", "gmail.com", "srmist.edu.in"]:
    r = check_temp_heuristics(d)
    icon = "⚠️" if r['is_temporary'] else "✅"
    print(f"{icon} {d:25s} → [{r['method']}] {r['message']}")

Testing: sr6172@srmist.edu.in
sr6172@srmist.edu.in is a valid and deliverable email!


In [ ]:
# =====================================================
# Layer 5 — Domain Age Check (WHOIS)
# =====================================================
import whois
from datetime import datetime, timezone as tz

def check_domain_age(domain: str) -> dict:
    """WHOIS lookup to determine domain registration age."""
    try:
        w = whois.whois(domain)
        creation = w.creation_date
        if isinstance(creation, list):
            creation = creation[0]
        registrar = w.registrar or 'Unknown'

        if creation:
            if creation.tzinfo is None:
                creation = creation.replace(tzinfo=tz.utc)
            age_days = (datetime.now(tz.utc) - creation).days
            if age_days < 30:
                risk, msg = 'high', f'Only {age_days} days old — very new'
            elif age_days < 90:
                risk, msg = 'medium', f'{age_days} days old — relatively new'
            elif age_days < 365:
                risk, msg = 'low', f'{age_days} days old — less than a year'
            else:
                risk, msg = 'safe', f'~{age_days // 365} year(s) old — established'
            return {'age_days': age_days, 'creation_date': str(creation),
                    'registrar': registrar, 'risk_level': risk, 'message': msg}
        return {'age_days': None, 'risk_level': 'unknown', 'message': 'Creation date missing'}
    except Exception as exc:
        return {'age_days': None, 'risk_level': 'unknown', 'message': f'WHOIS unavailable: {exc}'}


# Quick test
for d in ["gmail.com", "srmist.edu.in", "outlook.com"]:
    r = check_domain_age(d)
    icon = "🟢" if r['risk_level'] == 'safe' else "🟡" if r['risk_level'] in ('low', 'unknown') else "🔴"
    print(f"{icon} {d:25s} → {r['message']}")
    if r.get('registrar'):
        print(f"   Registrar: {r['registrar']}")

=== Real-Time Email Validation Test ===

Testing: sr6172@srmist.edu.in
Result: ✅ VALID
--------------------------------------------------

Testing: user@gm.com
Result: ✅ VALID
--------------------------------------------------

Testing: invalid@fake-domain.xyz
No MX record found: The DNS query name does not exist: fake-domain.xyz.
Failed MX check.
Result: ❌ INVALID
--------------------------------------------------

Testing: bad-format@
Failed regex check.
Result: ❌ INVALID
--------------------------------------------------

Testing: test@yahoo.com
Result: ✅ VALID
--------------------------------------------------

Testing: user@outlook.com
Result: ✅ VALID
--------------------------------------------------

Testing: dadqe@sadswda.com
No MX record found: The DNS query name does not exist: sadswda.com.
Failed MX check.
Result: ❌ INVALID
--------------------------------------------------


In [ ]:
# =====================================================
# Layer 6 — SPF Record Check
# =====================================================
import dns.resolver

def check_spf(domain: str) -> dict:
    """Query TXT records for SPF (v=spf1)."""
    try:
        answers = dns.resolver.resolve(domain, 'TXT')
        for rdata in answers:
            txt = rdata.to_text().strip('"')
            if txt.lower().startswith('v=spf1'):
                strictness = 'unknown'
                if '-all' in txt:    strictness = 'strict'
                elif '~all' in txt:  strictness = 'softfail'
                elif '?all' in txt:  strictness = 'neutral'
                elif '+all' in txt:  strictness = 'permissive'
                return {'found': True, 'record': txt, 'strictness': strictness,
                        'message': f'SPF found ({strictness})'}
    except Exception:
        pass
    return {'found': False, 'record': None, 'strictness': None,
            'message': 'No SPF record found'}

# Quick test
for d in ["gmail.com", "outlook.com", "yahoo.com", "srmist.edu.in"]:
    r = check_spf(d)
    icon = "✅" if r['found'] else "❌"
    print(f"{icon} {d:20s} → {r['message']}")
    if r['record']:
        print(f"   Record: {r['record'][:80]}…" if len(r.get('record','')) > 80 else f"   Record: {r['record']}")

In [ ]:
# =====================================================
# Layer 7 — DKIM Record Check
# =====================================================
DKIM_SELECTORS = [
    'default', 'google', 'selector1', 'selector2',
    'k1', 'dkim', 'mail', 's1', 's2', 'mx', 'email',
]

def check_dkim(domain: str) -> dict:
    """Probe common DKIM selectors for a public key."""
    for selector in DKIM_SELECTORS:
        try:
            qname = f'{selector}._domainkey.{domain}'
            answers = dns.resolver.resolve(qname, 'TXT')
            for rdata in answers:
                txt = rdata.to_text().strip('"')
                if 'p=' in txt:
                    return {'found': True, 'selector': selector,
                            'message': f'DKIM key found (selector: {selector})'}
        except Exception:
            continue
    return {'found': False, 'selector': None,
            'message': 'No DKIM record for common selectors'}

# Quick test
for d in ["gmail.com", "outlook.com", "yahoo.com"]:
    r = check_dkim(d)
    icon = "✅" if r['found'] else "❌"
    print(f"{icon} {d:20s} → {r['message']}")

In [ ]:
# =====================================================
# Layer 8 — DMARC Policy Check
# =====================================================

def check_dmarc(domain: str) -> dict:
    """Query _dmarc.<domain> TXT for DMARC policy."""
    try:
        answers = dns.resolver.resolve(f'_dmarc.{domain}', 'TXT')
        for rdata in answers:
            txt = rdata.to_text().strip('"')
            if 'v=dmarc1' in txt.lower():
                policy = 'none'
                m = re.search(r'p\s*=\s*(reject|quarantine|none)', txt, re.I)
                if m:
                    policy = m.group(1).lower()
                return {'found': True, 'record': txt, 'policy': policy,
                        'message': f'DMARC found: p={policy}'}
    except Exception:
        pass
    return {'found': False, 'record': None, 'policy': None,
            'message': 'No DMARC record found'}

# Quick test
for d in ["gmail.com", "outlook.com", "yahoo.com", "srmist.edu.in"]:
    r = check_dmarc(d)
    icon = "✅" if r['found'] else "❌"
    print(f"{icon} {d:20s} → {r['message']}")
    if r['record']:
        print(f"   {r['record'][:90]}")

In [ ]:
# =====================================================
# Layer 9 — MX / DNS Deliverability Check
# =====================================================
import time

def check_mx(domain: str) -> dict:
    """MX record lookup with A-record fallback."""
    start = time.time()
    try:
        mx_answers = dns.resolver.resolve(domain, 'MX')
        elapsed = round((time.time() - start) * 1000, 2)
        records = sorted(
            [{'priority': r.preference, 'exchange': str(r.exchange).rstrip('.')}
             for r in mx_answers],
            key=lambda r: r['priority'],
        )
        return {
            'has_mx': True, 'mx_count': len(records), 'mx_records': records,
            'response_time_ms': elapsed,
            'message': f'{len(records)} MX record(s) found — domain can receive email',
        }
    except dns.resolver.NXDOMAIN:
        return {'has_mx': False, 'mx_count': 0, 'mx_records': [],
                'message': 'Domain does not exist (NXDOMAIN)'}
    except dns.resolver.Timeout:
        return {'has_mx': None, 'mx_count': 0, 'mx_records': [],
                'message': 'DNS lookup timed out'}
    except Exception as exc:
        # Try A-record fallback
        try:
            a = dns.resolver.resolve(domain, 'A')
            return {'has_mx': False, 'has_a_record': True, 'mx_count': 0, 'mx_records': [],
                    'fallback_ips': [str(r) for r in a],
                    'message': 'No MX records; A record exists (RFC fallback)'}
        except Exception:
            return {'has_mx': False, 'mx_count': 0, 'mx_records': [],
                    'message': f'No MX or A records — cannot receive email'}


# Quick test
for d in ["gmail.com", "yahoo.com", "srmist.edu.in", "fake-nonexistent-domain-xyz.com"]:
    r = check_mx(d)
    icon = "✅" if r['has_mx'] else "❌"
    print(f"{icon} {d:35s} → {r['message']}")
    for mx in r.get('mx_records', [])[:3]:
        print(f"   Priority {mx['priority']:3d}  {mx['exchange']}")

In [ ]:
# =====================================================
# Weighted Risk-Scoring Engine (0–100)
# =====================================================
# Weights:
#   Blacklist 30% | Domain age 15% | SPF 15% | DKIM 10%
#   DMARC 10%     | MX 10%         | Heuristics 10%

WEIGHTS = {
    'blacklist':  0.30,
    'domain_age': 0.15,
    'spf':        0.15,
    'dkim':       0.10,
    'dmarc':      0.10,
    'mx':         0.10,
    'heuristics': 0.10,
}

def calculate_risk_score(checks: dict) -> dict:
    """Compute weighted risk score from all layer results."""
    scores = {}

    # Blacklist
    scores['blacklist'] = 100 if checks.get('blacklist', {}).get('is_blacklisted') else 0

    # Domain age
    age = checks.get('domain_age', {})
    age_days = age.get('age_days')
    if age_days is None:
        scores['domain_age'] = 50
    elif age_days < 30:
        scores['domain_age'] = 90
    elif age_days < 90:
        scores['domain_age'] = 60
    elif age_days < 365:
        scores['domain_age'] = 25
    else:
        scores['domain_age'] = 5

    # SPF
    spf = checks.get('spf', {})
    if not spf.get('found'):
        scores['spf'] = 80
    elif spf.get('strictness') == 'strict':
        scores['spf'] = 0
    elif spf.get('strictness') == 'softfail':
        scores['spf'] = 30
    else:
        scores['spf'] = 60

    # DKIM
    scores['dkim'] = 0 if checks.get('dkim', {}).get('found') else 70

    # DMARC
    dmarc = checks.get('dmarc', {})
    if not dmarc.get('found'):
        scores['dmarc'] = 80
    elif dmarc.get('policy') == 'reject':
        scores['dmarc'] = 0
    elif dmarc.get('policy') == 'quarantine':
        scores['dmarc'] = 30
    else:
        scores['dmarc'] = 60

    # MX
    mx = checks.get('mx', {})
    scores['mx'] = 0 if mx.get('has_mx') else 100

    # Heuristics
    heur = checks.get('heuristics', {})
    scores['heuristics'] = heur.get('confidence', 0) if heur.get('is_temporary') else 0

    # Weighted sum
    risk = round(sum(scores[k] * WEIGHTS[k] for k in WEIGHTS), 1)
    risk = min(max(risk, 0), 100)

    level = ('critical' if risk >= 80 else 'high' if risk >= 60
             else 'medium' if risk >= 40 else 'low' if risk >= 20 else 'safe')

    return {'risk_score': risk, 'risk_level': level, 'component_scores': scores}


print("Risk scoring engine defined ✅")
print(f"Weights: {WEIGHTS}")

In [ ]:
# =====================================================
# Full 9-Layer Pipeline Orchestrator
# =====================================================

def validate_email_comprehensive(email: str) -> dict:
    """Run all 9 layers and compute composite risk score."""
    start = time.time()
    result = {
        'email': email, 'is_valid': False,
        'is_temporary': False, 'is_deliverable': False,
    }

    # Layer 1: Format
    fmt = check_format(email)
    result['format'] = fmt
    if not fmt['is_valid']:
        result['processing_time_ms'] = round((time.time() - start) * 1000, 2)
        return result

    # Layer 2: Library
    lib = check_library(email)
    result['library'] = lib
    if not lib['is_valid']:
        result['processing_time_ms'] = round((time.time() - start) * 1000, 2)
        return result

    domain = lib.get('domain') or email.split('@')[1].lower()
    result['domain'] = domain
    result['is_valid'] = True

    # Layer 3: Blacklist
    bl = check_blacklist(domain)
    result['blacklist'] = bl
    if bl['is_blacklisted']:
        result['is_temporary'] = True

    # Layer 4: Heuristics
    heur = check_temp_heuristics(domain)
    result['heuristics'] = heur
    if heur['is_temporary'] and not bl['is_blacklisted']:
        result['is_temporary'] = True

    # Layer 5: Domain age
    age = check_domain_age(domain)
    result['domain_age'] = age

    # Layer 6: SPF
    spf = check_spf(domain)
    result['spf'] = spf

    # Layer 7: DKIM
    dkim = check_dkim(domain)
    result['dkim'] = dkim

    # Layer 8: DMARC
    dmarc = check_dmarc(domain)
    result['dmarc'] = dmarc

    # Layer 9: MX
    mx = check_mx(domain)
    result['mx'] = mx
    result['is_deliverable'] = mx.get('has_mx') is True

    # Risk scoring
    risk = calculate_risk_score(result)
    result['risk_score'] = risk['risk_score']
    result['risk_level'] = risk['risk_level']
    result['component_scores'] = risk['component_scores']
    result['confidence'] = round(100 - risk['risk_score'], 1)

    result['processing_time_ms'] = round((time.time() - start) * 1000, 2)
    return result


print("Full pipeline orchestrator defined ✅")

In [ ]:
# =====================================================
# 🧪 Comprehensive Test Suite
# =====================================================

test_emails = [
    "sr6172@srmist.edu.in",       # Valid college email
    "user@gmail.com",             # Valid Gmail
    "test@netoiu.com",            # Disposable (blocklist)
    "abc@mailinator.com",         # Disposable (blocklist)
    "user@outlook.com",           # Valid Outlook
    "hello@tempmail.xyz",         # Temp heuristic
    "invalid@fake-domain.xyz",    # Likely undeliverable
    "bad-format@",                # Invalid format
    "not-an-email",               # No @ sign
]

print("=" * 80)
print(f"{'EMAIL VALIDATION — 9-LAYER TEST SUITE':^80}")
print("=" * 80)

for email in test_emails:
    print(f"\n📧 Testing: {email}")
    print("-" * 60)
    r = validate_email_comprehensive(email)

    if not r['is_valid']:
        print(f"  ❌ INVALID — {r.get('format', r.get('library', {})).get('message', 'Failed early checks')}")
        print(f"  ⏱️  {r['processing_time_ms']}ms")
        continue

    # Status line
    status_parts = []
    if r['is_valid']:        status_parts.append("✅ Valid")
    if r['is_temporary']:    status_parts.append("🚫 Temporary")
    if r['is_deliverable']:  status_parts.append("📬 Deliverable")
    else:                    status_parts.append("📭 Not Deliverable")

    print(f"  {' | '.join(status_parts)}")
    print(f"  Risk: {r['risk_score']}/100 ({r['risk_level'].upper()}) | Confidence: {r['confidence']}%")

    # Layer results
    bl = r.get('blacklist', {})
    print(f"  Blocklist: {'🚫 LISTED' if bl.get('is_blacklisted') else '✅ Clean'}")

    age = r.get('domain_age', {})
    if age.get('age_days') is not None:
        print(f"  Domain age: {age['age_days']} days ({age['risk_level']})")

    print(f"  SPF: {'✅' if r.get('spf',{}).get('found') else '❌'}  "
          f"DKIM: {'✅' if r.get('dkim',{}).get('found') else '❌'}  "
          f"DMARC: {'✅' if r.get('dmarc',{}).get('found') else '❌'}")

    mx = r.get('mx', {})
    print(f"  MX records: {mx.get('mx_count', 0)}")

    print(f"  ⏱️  {r['processing_time_ms']}ms")

print("\n" + "=" * 80)
print("Done ✅")

---

## Architecture Summary

```
Email Input
    │
    ├─── Layer 1: Regex Format ──────────── fail? → STOP
    ├─── Layer 2: email-validator lib ───── fail? → STOP
    │
    ├─── Layer 3: Blocklist (5,100+) ───── disposable? → flag
    ├─── Layer 4: Heuristics ───────────── temp pattern? → flag
    ├─── Layer 5: WHOIS Domain Age ─────── < 30 days? → high risk
    ├─── Layer 6: SPF Record ───────────── missing? → risk ↑
    ├─── Layer 7: DKIM Signature ───────── missing? → risk ↑
    ├─── Layer 8: DMARC Policy ─────────── missing? → risk ↑
    └─── Layer 9: MX / DNS ────────────── no MX? → not deliverable
                    │
                    ▼
          Risk Scoring Engine (0–100)
          ┌──────────────────────────────┐
          │ Blacklist      30%           │
          │ Domain Age     15%           │
          │ SPF            15%           │
          │ DKIM           10%           │
          │ DMARC          10%           │
          │ MX             10%           │
          │ Heuristics     10%           │
          └──────────────────────────────┘
                    │
                    ▼
          Final Verdict + Confidence Score
```

### Django Integration
The production version in `App/EmailValidation/views.py` additionally includes:
- **Database caching** for WHOIS + DNS auth results (7-day TTL)
- **Behavioral monitoring** — per-IP rate limiting, bulk-temp-check detection, domain-abuse flags
- **3 Django models**: `EmailValidationLog`, `DomainCache`, `BehavioralFlag`
- **REST API** at `/emailvalidation/api/validate/`